In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Úprava surových dat: anonymizace

In [ ]:
df = pd.read_csv("data/input/respondents.csv", delimiter='|')
df = df.drop(columns=['first_name', 'last_name'])
df.to_csv('data/respondents.csv', sep='|')

### Načtení anonymizovaných dat 

In [2]:
df = pd.read_csv("data/respondents.csv", delimiter='|', index_col=0)
df.head()

,id,party_name,descriptions,gender,is_foreigner,location
0,00000023-93ee-4754-a3c4-5748a4f420ad,STAN,hasič;Všeruby (Domažlice);ex-zast_obec;ex-kand...,1,2,554456
1,0000d8c9-28cf-457b-a1ff-751e3d1fac3c,BEZPP,Potravinářská komora České republiky;lobby;mluvčí,2,2,0
2,00017467-20c1-4151-9126-088f9bff86e9,BEZPP,umělec;dirigent,1,2,0
3,0002bfa9-c1e6-4fae-acfd-6e32481e7fd5,BEZPP,umělec;básnířka,2,2,0
4,00040844-ca87-4a12-b394-eecd78a0166f,BEZPP,mluvčí Rožnov pod Radhoštěm;umělec;grafička a ...,2,2,0


In [3]:
df[df['descriptions'].str.contains('-') & (~df['descriptions'].str.contains('ex-'))].descriptions

14       ředitel odboru administrativně-právního ČOI;ko...
23       oddělení tisku a prevence PČR Benešov;OM_ID (3...
65       odborník;kurátorka pavilonu pand;vedoucí chovu...
87       zdravotník;lékař -intervenční radiolog FN Olom...
101      Právník, specialista na dopravně-správní agned...
                               ...                        
81543    Policie;oddělení tisku a prevence;OM_ID (141DB...
81549    OM_ID (40ADB60A-EBB5-4F9F-A6EB-962D35AF);staro...
81557    náměstek hejtmana Pardubického kraje pověřený ...
81571      social;pracovnice nouzového call-centra v Mekce
81615    kastelán;kultura;správce dolu Michal;Důl Micha...
Name: descriptions, Length: 3685, dtype: object

In [4]:
pd.set_option('display.max_colwidth', None)
df[df['descriptions'].str.contains('A-E')]

,id,party_name,descriptions,gender,is_foreigner,location
2289,0dd661cf-2e36-4146-85de-d62b20b88d0c,BEZPP,mluvčí firmy Sovis;OM_ID (8F34FA9A-EE05-4E26-B933-62CB8E97);byznys,2,2,0
7167,2bbf0af7-277a-42ff-939a-9688dd1e9e3b,BEZPP,psycholog;odborník;OM_ID (E31020EA-0E19-4AA5-A8DA-E99069AE);OM_ID (876CF489-483D-4CB7-8913-E1FC498D),1,2,0
7752,2f64fd93-9282-40fc-9b77-80ce2517ec35,BEZPP,OM_ID (AFA3C333-6913-4FAA-B32A-E9814CF7);Vedoucí odboru sociálních věcí;social;úředník,2,2,0
8951,37276222-ba11-4320-8f06-f7f9addcf804,BEZPP,OM_ID (CC9BEF7A-E049-476D-A63A-61A4FB3E);atletka;sport,2,2,0
19233,7747a945-a33c-4120-9f07-93a491c46b3a,BEZPP,akademik;OM_ID (BEC147AA-E5DB-43F5-B77C-BFC45C56);vědkyně,2,2,0
21584,85ba095b-2cb9-4772-b53c-7226838a94c6,BEZPP,OM_ID (A69A7A5C-40A8-4629-B4CA-E18E5C6D);akademik;VŠCHT;chemik,1,2,0
25222,9ca19abc-421b-4743-9a97-b5875ba34c6f,BEZPP,Mobilní hospic Ondrášek;ekonomka Mobilního hospice Ondrášek;OM_ID (A541900A-E9F9-4F8F-B9E2-DC448193);social;vedoucí úseku péče o dárce,1,2,0
26897,a71118ef-5aa9-4039-b6b9-f9c046ab0ab1,BEZPP,plavec;sport;OM_ID (19D0F0F6-48F1-4408-A84A-EB889474),1,2,0
36106,e2bb6de9-430f-45be-9364-0058ef0b59dd,BEZPP,Nesáčkuj;OM_ID (DEC90E75-95F3-4BC5-8A0A-E2D5C74C);aktivista;ngo;autorka projektu,2,2,0
42137,090b9182-701f-4733-b9d4-14ae3fad9ac8,ZAHR,konzul;vyslanec;OM_ID (A824DD4A-E903-4677-A2E1-2A68EAD8),1,1,0


In [5]:
df[df['descriptions'].str.contains('ID') & (~df.descriptions.str.contains('OM_ID'))]

,id,party_name,descriptions,gender,is_foreigner,location
1650,09f2ea98-f3a8-40f6-bd93-d0b7d5c9de2e,BEZPP,byznys;jednatel IDPS,1,2,0
2144,0ccdea37-f412-4673-b09f-37ac792e5fd3,BEZPP,mluvčí;mluvčí VIDA!,2,2,0
3029,12015352-b032-4a52-80c1-ac983442704d,ML,zdravotník;lékař;primář GPO Sokolov;Mariánské Lázně;ex-kand_obec;ex-kandidát za MĚSTO LIDEM,1,2,554642
4712,1c90dec4-d384-4806-9abb-24139f2ea734,BEZPP,koordinátor konsorcia UNIDA;social,1,2,0
5257,200ace97-e04e-4cbd-bac2-b29c99156e61,KDU-ČSL,"pedagog, zastupitel MČ Praha 6;kand_posl;učitel;ex-zast_obec;ex-kand_obec;ex-kandidát za KLID. Koalice TOP 09 a KDU-ČSL",1,2,500178
...,...,...,...,...,...,...
75155,d7255db2-bee5-4a00-9b02-5e26dd60aad5,BEZPP,mluvčí;ROPID;mluvčí ROPIDu;mluvčí společnosti,1,2,0
77870,e85bcb8c-0b8f-46d2-a686-db5646e1049d,BEZPP,Odbor inteligentních dopravních systémů ministerstvo dopravy;ředitel odboru IDS MD;úředník,1,2,0
78047,e97b0c7b-e55e-4ba6-b1a6-2312314d8013,KDU-ČSL,starosta města;kand_posl;starosta Světlé nad Sázavou;ex-zast_obec;ex-kand_obec;ex-kandidát za KDU-ČSL a NEZÁVISLÍ KANDIDÁTI,1,2,569569
80424,f8835599-c371-4783-9371-f39223e786a0,BEZPP,jihomoravský krajský předseda Sdružení praktických lékařů;spolek;lékař;Česká společnost pro AIDS pomoc;zdravotník,1,2,0


In [6]:
df[df['descriptions'].str.contains('kand_')]

,id,party_name,descriptions,gender,is_foreigner,location
0,00000023-93ee-4754-a3c4-5748a4f420ad,STAN,"hasič;Všeruby (Domažlice);ex-zast_obec;ex-kandidát za """"ZA ROZVOJ OBCE"""";ex-kand_obec",1,2,554456
15,0015cb76-f7c4-468f-b112-f5ad034f3a3b,KDU-ČSL,REG_ZPR (#4911);středoškolský pedagog;exTOP 09;SŠ pedagožka;exposlankyně;odborník;kand_posl_2021;předsedkyně Středoevropské inspirace;ex-zast_obec;ex-kand_europol;ex-kandidát za Piráti se STANem;ex-kand_obec,2,2,538094
17,001af0bb-9287-4dc9-baee-69c5f31a8184,Patrioti ČR,tlumočnice;překladatelka;ex-kand_europol,2,2,0
18,001dce28-2ee8-438d-abe4-09c52f3ba75a,TOP 09,kand_posl;veterinární lékař;odborník;soukromý veterinární lékař;veterinář;ex-kand_obec;ex-kandidát za TOP 09,1,2,537454
29,002a0b1f-d798-4f40-8713-d858ded28edb,BEZPP,fotbalista;tiskový mluvčí;sport;Pardubice VI;ex-kand_obec;ex-kandidát za Pardubice pro lidi,1,2,0
...,...,...,...,...,...,...
81574,ffb8b054-a874-4e67-90c5-84fc8238ab4c,ADS,grafik a student;ex-kand_europol,2,2,0
81575,ffbac3a6-accf-4b2e-a40b-cd3d5d9d576a,ČSSD,zastupitel Brna-Králova Pole;specialista v oblasti BOZP a PO;specialista v oblasti bezpečnosti práce a PO;ex-zast_obec;ex-kand_obec;ex-kandidát za ČSSD,1,2,551007
81607,ffe612f9-8e1e-4cd0-9883-b30b5b33c213,ODS,"kand_posl;auditor;daňový poradce;ex-zast_obec;ex-kand_obec;ex-kandidát za ODS, TOP 09, SOUKROMNÍCI A NK",1,2,505927
81618,fff9569f-106f-48b9-98f4-6466ad5d2909,Piráti,sítotiskař;OSVČ;kand_posl;ex-zast_obec;ex-kand_obec;ex-kandidát za Piráti,1,2,555088


In [7]:
df[df['descriptions'].str.contains("'")]

,id,party_name,descriptions,gender,is_foreigner,location
32013,c8342a83-44c3-49be-a4b1-475096316065,BEZPP,Baha'i;sport;cvičitelka jógy,2,2,0
67334,a56e3994-9482-48f4-8376-4a9cf816a737,BEZPP,umělec;novinář;šéfredaktorka Harper's Bazaar;spisovatelka,2,2,0
79290,f1ac8050-83ff-4190-947d-9f8d97dc07af,BEZPP,byznys;McDonald's,2,2,0


In [8]:
import json
import subprocess
import requests
from pandarallel import pandarallel
pandarallel.initialize()

zkratky = {'zast': 'zastupitel', 'kand': 'kandidát', 'posl': 'poslanec'}
main_separator = ';'
interpunkcni_z = '",.;:_!?(){}-'
stopword_file = 'data/stopwords-cs.json'
political_parties = ['ods', 'kdu', 'čsl', 'čssd', 'ano', 'piráti', 'stan', 'ksčm', 'spd', 'top 09', 'ano 2011']

def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

def replace_party(word: str) -> str:
    if word in political_parties:
        return "strana"
    return word

def unify_parties(column: pd.Series) -> pd.Series:
    return column.apply(lambda row: [replace_party(x) for x in row])

def remove_shortened(word: str) -> str:
    if word in zkratky.keys():
        return zkratky[word]
    return word

def remove_shortened_words(column: pd.Series, translations: dict = zkratky) -> pd.Series:
    return column.apply(lambda row: [remove_shortened(x) for x in row])

def remove_interpunctions(column: pd.Series, interpunkcni_z: str = interpunkcni_z, sep: str =' ') -> pd.Series:
    without_interpunction = column
    for letter in interpunkcni_z:
        without_interpunction = without_interpunction.str.replace(letter, sep)
    return without_interpunction

def split_into_words(column: pd.Series, sep=' ') -> pd.Series:
    return column.str.split(sep)

def remove_stop_words(column: pd.Series, filename: str = stopword_file) -> pd.Series:
    with open(filename, 'r') as fd:
        stop_words = json.load(fd)
        return column.apply(lambda words: [word for word in words if word not in stop_words])
def remove_empty(column: pd.Series) -> pd.Series:
    return column.apply(lambda words: [word for word in words if word != ''])
def remove_numbers(column: pd.Series) -> pd.Series:
    return column.apply(lambda words: [word for word in words if not has_numbers(word)])

def lemmatizate_word(word: str) -> str:
    word = bytes(word, encoding='utf-8')
    p = subprocess.Popen("lib/majka -f data/majka.w-lt| head -n1 | cut -d ':' -f1",
                         shell=True, stdout=subprocess.PIPE,
                         stderr=subprocess.PIPE,
                         stdin=subprocess.PIPE)
    lemma, _ = p.communicate(input=word)
    lemma = lemma.strip(b'\n')
    if len(lemma) == 0:
        lemma = word
    return lemma.decode(encoding='utf-8')

def lemmatizate_words(words: list) -> list:
    lemmatized = []
    for word in words:
        lemmatized.append(lemmatizate_word(word))
    return lemmatized

def lemmatizate(column: pd.Series) -> pd.Series:    
    return column.apply(lemmatizate_words)

def apply_pipeline(column: pd.Series, pipeline: list) -> pd.Series:
    processed = column.copy()
    for func in pipeline:
        processed = func(processed)
    return processed

def lower(column: pd.Series) -> pd.Series:
    return column.str.lower()

def correct_word(word: str) -> str:
    url = 'http://localhost/correct?data=' + word
    response = requests.get(url)
    corrected = response.json()['result']
    return response.json()['result']

def correct_grammar(column: pd.Series) -> pd.Series:
    return column.parallel_apply(lambda row: [correct_word(word) for word in row])

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [9]:
%%time
from lib.czech_stemmer.czech_stemmer import cz_stem
pipeline1 = [lower, remove_interpunctions, split_into_words, remove_empty, unify_parties, remove_shortened_words, remove_numbers,remove_stop_words]
pipeline2 = [lower, remove_interpunctions, split_into_words, remove_empty, unify_parties, remove_shortened_words, remove_numbers, correct_grammar, remove_stop_words]

result = apply_pipeline(df.descriptions, pipeline1)
result_corrected = apply_pipeline(df.descriptions, pipeline2)

CPU times: user 4.38 s, sys: 197 ms, total: 4.57 s
Wall time: 4min 34s


In [10]:
df.head()

,id,party_name,descriptions,gender,is_foreigner,location
0,00000023-93ee-4754-a3c4-5748a4f420ad,STAN,"hasič;Všeruby (Domažlice);ex-zast_obec;ex-kandidát za """"ZA ROZVOJ OBCE"""";ex-kand_obec",1,2,554456
1,0000d8c9-28cf-457b-a1ff-751e3d1fac3c,BEZPP,Potravinářská komora České republiky;lobby;mluvčí,2,2,0
2,00017467-20c1-4151-9126-088f9bff86e9,BEZPP,umělec;dirigent,1,2,0
3,0002bfa9-c1e6-4fae-acfd-6e32481e7fd5,BEZPP,umělec;básnířka,2,2,0
4,00040844-ca87-4a12-b394-eecd78a0166f,BEZPP,mluvčí Rožnov pod Radhoštěm;umělec;grafička a výtvarnice,2,2,0


In [11]:
from src.utils import to_bag_of_words
really_coarse_bag = to_bag_of_words(result, is_string=False)
really_coarse_bag.head()

,aaa,aaabyty,aaaf,aace,aadd,aag,aakrchitektura,aalborg,aarhus,ab,...,žďársko,žďársku,žďárská,žďárské,žďárského,žďáru,žďáře,žš,žžm,ˇyďár
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
coarse_bag = to_bag_of_words(result_corrected, is_string=False)
coarse_bag.head()

,aaa,aalborg,aarhus,abatyše,abda,abeceda,abertamy,about,abrahámovy,abs,...,žďas,žďár,žďárce,žďárem,žďársko,žďársku,žďárská,žďárské,žďárského,žďáru
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
i = 0
def get_row_roots(row: list) -> list:
    if len(row) == 0:
        return row
    words = " ".join(row)
    #print(words)
    url = f'http://localhost:3000/analyze?data={words}&derivation=root&output=vertical&convert_tagset=strip_lemma_id'
    response = requests.get(url)
    return [x.split('\t')[1] for x in response.json()['result'].split('\n') if len(x) != 0]

def get_roots(column: pd.Series) -> pd.Series:   
    return column.parallel_apply(get_row_roots)

In [40]:
pandarallel.initialize()
roots = get_roots(result_corrected)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [42]:
df['roots'] = roots

In [44]:
to_bag_of_words(roots, is_string=False).head()

,aaa,aalborg,aarhus,abatyše,abd,abeceda,abertamy,about,abraham,absint,...,žába,žádník,žák,žár,žíla,žít,žítek,žížala,žďas,žďára
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
%%time
lemmatized_corrected = result_corrected.parallel_apply(lemmatizate_words)

CPU times: user 4min 47s, sys: 33.1 s, total: 5min 20s
Wall time: 5min 21s


In [49]:
lemmatized = result.parallel_apply(lemmatizate_words)

In [50]:
to_bag_of_words(lemmatized_corrected, is_string=False).head()

,aaa,aalborg,aarhus,abatyše,abda,abeceda,abertamy,about,abrahámovy,abs,...,žár,žít,žítkové,žížala,žďas,žďár,žďárce,žďársko,žďársku,žďárský
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
to_bag_of_words(lemmatized, is_string=False)

,aaa,aaabyty,aaaf,aace,aadd,aag,aakrchitektura,aalborg,aarhus,ab,...,žďár,žďárce,žďárku,žďársko,žďársku,žďárský,žďáře,žš,žžm,ˇyďár
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81620,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81621,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81622,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81623,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
stemmed_l = lemmatized.apply(lambda x: [cz_stem(word) for word in x]) 
stemmed_corrected_l = lemmatized_corrected.apply(lambda x: [cz_stem(word) for word in x]) 

stemmed_a = lemmatized.apply(lambda x: [cz_stem(word, aggressive=True) for word in x])
stemmed_corrected_a = lemmatized_corrected.apply(lambda x: [cz_stem(word, aggressive=True) for word in x])

In [53]:
to_bag_of_words(stemmed_l, is_string=False).head()

,aaa,aaabyt,aaaf,aadd,aag,aak,aakrchitektur,aalborg,aarh,ab,...,žížal,žďas,žďánic,žďár,žďárk,žďársk,žďář,žš,žžm,ˇyďár
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
to_bag_of_words(stemmed_corrected_l, is_string=False).head()

,aaa,aalborg,aarh,abatyš,abd,abeced,abertam,about,abrahám,abs,...,žák,žákov,žár,žít,žítk,žížal,žďas,žďár,žďárk,žďársk
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
to_bag_of_words(stemmed_a, is_string=False).head()

,aaa,aaaby,aaaf,aadd,aag,aak,aakrchitektur,aalborg,aarh,ab,...,žít,žíža,žďas,žďán,žďár,žďárs,žďář,žš,žžm,ˇyďár
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
to_bag_of_words(stemmed_corrected_a, is_string=False).head()

,aaa,aalborg,aarh,abatyš,abd,abeced,abertam,abou,abrahám,abs,...,žurná,žáb,žád,žák,žár,žít,žíža,žďas,žďár,žďárs
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
df['descr_procc'] = lemmatized_corrected
df['descr_procc_l'] = stemmed_corrected_l
df['descr_procc_a'] = stemmed_corrected_a
df.head()

,id,party_name,descriptions,gender,is_foreigner,location,roots,lemmatized_corrected,descr_procc,descr_procc_l,descr_procc_a
0,00000023-93ee-4754-a3c4-5748a4f420ad,STAN,"hasič;Všeruby (Domažlice);ex-zast_obec;ex-kandidát za """"ZA ROZVOJ OBCE"""";ex-kand_obec",1,2,554456,"[hasit, Všeruby, Domažlice, ex, zastoupit, obec, ex, kandidát, rozvoj, obec, ex, kandidát, obec]","[hasič, Všeruby, Domažlice, ex, zastupitel, obec, ex, kandidát, rozvoj, obec, ex, kandidát, obec]","[hasič, Všeruby, Domažlice, ex, zastupitel, obec, ex, kandidát, rozvoj, obec, ex, kandidát, obec]","[hasič, Všerub, Domažlik, ex, zastupitel, obec, ex, kandidát, rozvoj, obec, ex, kandidát, obec]","[hasi, Všerub, Domaž, ex, zastup, obe, ex, kandidá, rozvoj, obe, ex, kandidá, obe]"
1,0000d8c9-28cf-457b-a1ff-751e3d1fac3c,BEZPP,Potravinářská komora České republiky;lobby;mluvčí,2,2,0,"[trávit, komora, Čech, republika, lobby, mluvit]","[potravinářský, komora, český, republika, lobby, mluvčí]","[potravinářský, komora, český, republika, lobby, mluvčí]","[potravinářsk, komor, česk, republik, lobb, mluvč]","[potravinářs, komor, čes, repub, lobb, mluv]"
2,00017467-20c1-4151-9126-088f9bff86e9,BEZPP,umělec;dirigent,1,2,0,"[umět, dirigovat]","[umělec, dirigent]","[umělec, dirigent]","[umělec, dirigent]","[uměl, dirigen]"
3,0002bfa9-c1e6-4fae-acfd-6e32481e7fd5,BEZPP,umělec;básnířka,2,2,0,"[umět, bát]","[umělec, básnířka]","[umělec, básnířka]","[umělec, básnířk]","[uměl, básn]"
4,00040844-ca87-4a12-b394-eecd78a0166f,BEZPP,mluvčí Rožnov pod Radhoštěm;umělec;grafička a výtvarnice,2,2,0,"[mluvit, Rožnov, Radhošť, umět, karta, tvořit]","[mluvčí, Rožnov, Radhošť, umělec, grafička, výtvarnice]","[mluvčí, Rožnov, Radhošť, umělec, grafička, výtvarnice]","[mluvč, Rožn, Radhošť, umělec, grafičk, výtvarnik]","[mluv, Rož, Radhošť, uměl, gra, výtvar]"


In [58]:
df.to_csv('data/proccessed.csv')
df.head()

,id,party_name,descriptions,gender,is_foreigner,location,roots,lemmatized_corrected,descr_procc,descr_procc_l,descr_procc_a
0,00000023-93ee-4754-a3c4-5748a4f420ad,STAN,"hasič;Všeruby (Domažlice);ex-zast_obec;ex-kandidát za """"ZA ROZVOJ OBCE"""";ex-kand_obec",1,2,554456,"[hasit, Všeruby, Domažlice, ex, zastoupit, obec, ex, kandidát, rozvoj, obec, ex, kandidát, obec]","[hasič, Všeruby, Domažlice, ex, zastupitel, obec, ex, kandidát, rozvoj, obec, ex, kandidát, obec]","[hasič, Všeruby, Domažlice, ex, zastupitel, obec, ex, kandidát, rozvoj, obec, ex, kandidát, obec]","[hasič, Všerub, Domažlik, ex, zastupitel, obec, ex, kandidát, rozvoj, obec, ex, kandidát, obec]","[hasi, Všerub, Domaž, ex, zastup, obe, ex, kandidá, rozvoj, obe, ex, kandidá, obe]"
1,0000d8c9-28cf-457b-a1ff-751e3d1fac3c,BEZPP,Potravinářská komora České republiky;lobby;mluvčí,2,2,0,"[trávit, komora, Čech, republika, lobby, mluvit]","[potravinářský, komora, český, republika, lobby, mluvčí]","[potravinářský, komora, český, republika, lobby, mluvčí]","[potravinářsk, komor, česk, republik, lobb, mluvč]","[potravinářs, komor, čes, repub, lobb, mluv]"
2,00017467-20c1-4151-9126-088f9bff86e9,BEZPP,umělec;dirigent,1,2,0,"[umět, dirigovat]","[umělec, dirigent]","[umělec, dirigent]","[umělec, dirigent]","[uměl, dirigen]"
3,0002bfa9-c1e6-4fae-acfd-6e32481e7fd5,BEZPP,umělec;básnířka,2,2,0,"[umět, bát]","[umělec, básnířka]","[umělec, básnířka]","[umělec, básnířk]","[uměl, básn]"
4,00040844-ca87-4a12-b394-eecd78a0166f,BEZPP,mluvčí Rožnov pod Radhoštěm;umělec;grafička a výtvarnice,2,2,0,"[mluvit, Rožnov, Radhošť, umět, karta, tvořit]","[mluvčí, Rožnov, Radhošť, umělec, grafička, výtvarnice]","[mluvčí, Rožnov, Radhošť, umělec, grafička, výtvarnice]","[mluvč, Rožn, Radhošť, umělec, grafičk, výtvarnik]","[mluv, Rož, Radhošť, uměl, gra, výtvar]"


In [59]:
df.descr_procc_l

0        [hasič, Všerub, Domažlik, ex, zastupitel, obec, ex, kandidát, rozvoj, obec, ex, kandidát, obec]
1                                                     [potravinářsk, komor, česk, republik, lobb, mluvč]
2                                                                                     [umělec, dirigent]
3                                                                                      [umělec, básnířk]
4                                                     [mluvč, Rožn, Radhošť, umělec, grafičk, výtvarnik]
                                                      ...                                               
81620                                                                               [sport, baseballist]
81621                              [jednatel, společnost, agres, spol, r, asanačn, podnik, Medl, byznys]
81622                                                                       [sport, kuželk, reprezentak]
81623                                  [Head, of, Strat